In [ ]:
"""
Created: 6/7

Purpose: To incorporate the synapses
into the neuron object

"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")

import datajoint_utils as du
from importlib import reload

In [3]:
# #configuring will include the adapters
# minnie,schema = du.configure_minnie_vm()

In [4]:
import neuron_utils as nru
import neuron_visualizations as nviz

# Downloading a Neuron to Play with

In [5]:
segment_id = 864691136194042326

In [ ]:
# original_mesh = du.fetch_segment_id_mesh(segment_id)

In [ ]:
# neuron_obj  = du.decomposition_with_spine_recalculation(segment_id,split_index = 0)

In [ ]:
# import synapse_utils as syu
# neuron_obj_syn = syu.add_synapses_to_neuron_obj(neuron_obj,
#                                                 segment_id = segment_id,
#                             validation = True,
#                             verbose  = True,
#                             #original_mesh = original_mesh,
#                             plot_valid_error_synapses = False,
#                             calculate_synapse_soma_distance = False,
#                             add_valid_synapses = True,
#                               add_error_synapses=False)

# Decompressing the Neuron to place with

In [6]:
import system_utils as su
original_mesh = su.decompress_pickle("original_mesh.pbz2")

In [7]:
neuron_obj = nru.decompress_neuron("./864691136194042326_0_25_axon_v5.pbz2",
                                  original_mesh=original_mesh)

Decompressing Neuron in minimal output mode...please wait


In [8]:
nviz.visualize_neuron(neuron_obj,
                     limb_branch_dict="all")

Recieved another instance of Neuron class in init -- so just copying data

 Working on visualization type: mesh

 Working on visualization type: skeleton
No Synapses to plot


In [9]:
neuron_obj_syn = neuron_obj

# The high degree function

In [ ]:
"""
Pseudocode: 


"""

In [12]:
import neuron_searching as ns
pre_filtering_limb_branch = ns.query_neuron_by_labels(neuron_obj,
                            matching_labels=["axon"])
nviz.plot_limb_branch_dict(neuron_obj,
                           pre_filtering_limb_branch)

Recieved another instance of Neuron class in init -- so just copying data

 Working on visualization type: mesh
No Synapses to plot


# finding all branches with at least 3 downstream branches

In [111]:
if pre_filtering_limb_branch is None:
    pre_filtering_limb_branch = neuron_obj.limb_branch_dict

    
skip_distance = 2000
min_skeletal_distance = 3000

high_degree_limb_branch = ns.query_neuron(neuron_obj,
                functions_list=["n_downstream_nodes_with_skip","skeletal_length"],
                query = f"(n_downstream_nodes_with_skip>2) and (skeletal_length>{min_skeletal_distance})",
                function_kwargs=dict(skip_distance=skip_distance),
               limb_branch_dict_restriction=pre_filtering_limb_branch)

nviz.plot_limb_branch_dict(neuron_obj,
                           high_degree_limb_branch)

Recieved another instance of Neuron class in init -- so just copying data

 Working on visualization type: mesh
No Synapses to plot


# Building the high degree matching

In [120]:
import axon_utils as au

"""
Purpose: To implement the high degree
branching filter using graph implementation
to compare the matches of all the candidates in 
a high degree branching area

Pseudocode: 
1) Get all downstream branhes (with an optional skip distance)
"""
limb_obj = neuron_obj[0]
branch_idx = 223
skip_distance = 3000
verbose = False
remove_short_thick_endnodes = True
short_thick_endnodes_to_remove = None
min_degree_to_resolve = 3

verbose = True


#1) Get all downstream branhes (with an optional skip distance)
downstream_branches = cnu.endnode_branches_of_branches_within_distance_downtream(limb_obj,
                                                          branch_idx,
                                                          skip_distance=skip_distance)

if verbose:
    print(f"downstream_branches = {downstream_branches}")

if remove_short_thick_endnodes:
    if short_thick_endnodes_to_remove is None:
        short_thick_endnodes_to_remove = au.short_thick_branches_from_limb(limb_obj,
                             verbose = False)
    downstream_branches = np.setdiff1d(downstream_branches,short_thick_endnodes_to_remove)
    
    if verbose:
        print(f"downstream_branches after remove_short_thick_endnodes = {downstream_branches}")
        
        
if len(downstream_branches) < min_degree_to_resolve:
    if verbose:
        print(f"Number of branches ({len(downstream_branches)}), aka branches_at_coord = {downstream_branches}) was less than min_degree_to_resolve ({min_degree_to_resolve}) so returning no error branches")
    return_value =  None,np.array([])
    
    
if max_degree_to_resolve_wide is not None:
        up_width = au.axon_width(limb_obj[upstream_branch])
        if up_width > max_degree_to_resolve_width_threshold:
            max_degree_to_resolve = max_degree_to_resolve_wide
            print(f"Changing max_degree_to_resolve = {max_degree_to_resolve_wide} because upstream width was {up_width} ")

downstream_branches = [225 226 227]
downstream_branches after remove_short_thick_endnodes = [225 226 227]


In [96]:
%%timeit
limb_obj.concept_network_directional

40.5 ns ± 0.0762 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [98]:
%%timeit
xu.reverse_DiGraph(limb_obj.concept_network_directional)

7.55 µs ± 190 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [99]:
%%prun
#1) Get all downstream branhes (with an optional skip distance)


In [ ]:
offset=1500,
comparison_distance = 2000,
worst_case_match_threshold = 65,

#args for width matching
width_diff_max = 75,#np.inf,#100,
width_diff_perc = 60,

#args for definite pairs
match_threshold = 45,
angle_buffer = 15,

max_degree_to_resolve = 6,
max_degree_to_resolve_wide = 8,

#args for picking the final winner
match_method = "best_match", #other option is "best_match"

kiss_check = True,
kiss_check_bbox_longest_side_threshold = 450,

In [67]:
[k for k in dir(nru) if "dist" in k]

['branch_skeletal_distance_from_soma',
 'branches_within_skeletal_distance',
 'distance_to_soma_from_coordinate_close_to_branch',
 'global_comparison_distance',
 'skeletal_distance_from_soma',
 'synapse_skeletal_distances_to_soma']

# Filtering the Neuron and then adding back the errors

In [ ]:
import proofreading_utils as pru
o_neuron, filtering_info = pru.apply_proofreading_filters_to_neuron(input_neuron = neuron_obj,
                                        filter_list = pru.v5_exc_filters(),
                                        plot_limb_branch_filter_with_disconnect_effect=False,
                                        plot_limb_branch_filter_away=False,
                                        plot_final_neuron=False,

                                        return_error_info=True,
                                         verbose=True,
                                        verbose_outline=True,
                                        return_limb_branch_dict_to_cancel = True)
nviz.visualize_neuron(o_neuron,limb_branch_dict="all")

In [ ]:
import neuron
o_neuron_cp = neuron.Neuron(o_neuron)
syu.add_synapses_to_neuron_obj(o_neuron_cp,
                            validation = True,
                            verbose  = True,
                            original_mesh = original_mesh,
                            plot_valid_error_synapses = False,
                            calculate_synapse_soma_distance = False,
                            add_valid_synapses = False,
                              add_error_synapses=True)
o_neuron_cp.n_synapses_total

# How to Query Synapses

In [ ]:
syu.synapses_df(neuron_obj)

ret_df = syu.query_synapses(neuron_obj,
query = "label == 'soma'",
verbose = True,
return_index = True)

ret_df